In [1]:
# Imported Modules
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

In [2]:
# Extracts video frames
def get_frames(video, frame_interval):
    '''Extracts one frame every `frame_interval` frames from the video.'''
    cap = cv2.VideoCapture(video)
    all_frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_num = 0

    while frame_num < total_frames:
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
        ret, frame = cap.read()
        all_frames.append(frame)
        frame_num += frame_interval
    
    return all_frames

# Get a frame every 10 seconds
frame_interval = 500
all_frames1 = get_frames("Scrabble Game 1.mp4", frame_interval)
all_frames2 = get_frames("Scrabble Game 2.mp4", frame_interval)
all_frames3 = get_frames("Scrabble Game 3.mp4", frame_interval)

In [3]:
# Isolates board
def filter_background(im):
    '''Filters the background leaving just the board'''
    im_filtered = im.copy()
    lower_blue = np.array([0, 0, 0])
    upper_blue = np.array([80, 255, 255])
    mask = cv2.inRange(im, lower_blue, upper_blue)
    im_filtered[(mask == 0),:] = (0,0,0)

    return im_filtered

def find_largest_contour(im_filtered):
    '''Find the largest contour in the image which will be the board'''

    # Change image to grayscale and threshold for best results
    im_gray = cv2.cvtColor(im_filtered, cv2.COLOR_RGB2GRAY)
    thresh_val_used, thresh_im = cv2.threshold(im_gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    contours, hierarchy = cv2.findContours(thresh_im, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Biggest contour will be board
    largest_contour = max(contours, key = cv2.contourArea)

    return largest_contour

def find_corners(largest_contour):
    '''Finds corners of board'''

    # Find corners of board
    epsilon = 0.04*cv2.arcLength(largest_contour, True)
    corners = cv2.approxPolyDP(largest_contour, epsilon, True)
    corners = corners.reshape(-1, 2)
    corners = np.unique(corners, axis = 0)  # Sometimes poly function finds more than 4 corners
    
    # Rearrange to match reference data
    rearranged_corners = np.zeros((4,2), dtype = int)
    s = corners.sum(axis = 1)
    rearranged_corners[0] = corners[np.argmin(s)]     # Top left
    rearranged_corners[3] = corners[np.argmax(s)]     # Bottom right
    diff = np.diff(corners, axis = 1)
    rearranged_corners[1] = corners[np.argmin(diff)]  # Top right
    rearranged_corners[2] = corners[np.argmax(diff)]  # Bottom left

    return rearranged_corners

def generate_warped_board(im, corners):
    '''Uses the corners to generate a warped 800x800 image of the board'''
    
    # Changing corners order to work best with functions
    corners = corners[[0, 1, 3, 2], :]
    corners = corners.astype(np.float32)
    transfer_size = np.array([[0, 0], [800, 0], [800, 800], [0, 800]], dtype = np.float32)

    # Generating warped perspective
    M = cv2.getPerspectiveTransform(corners, transfer_size)
    warped = cv2.warpPerspective(im, M, (800, 800))

    return warped

def get_scrabble_board(all_frames):
    '''Isolates the scrabble board from the frames'''
    all_im = []
    all_im_filtered = []
    all_warped = []
    for i in range(len(all_frames)):
        im = cv2.cvtColor(all_frames[i], cv2.COLOR_BGR2RGB)
        im_filtered = filter_background(im)
        largest_contour = find_largest_contour(im_filtered)
        rearranged_corners = find_corners(largest_contour)
        warped = generate_warped_board(im, rearranged_corners)

        all_im.append(im)
        all_im_filtered.append(im_filtered)
        all_warped.append(warped)

    return all_im, all_im_filtered, all_warped  

# Get scrabble board data
all_im1, all_im_filtered1, all_warped1 = get_scrabble_board(all_frames1)
all_im2, all_im_filtered2, all_warped2 = get_scrabble_board(all_frames2)
all_im3, all_im_filtered3, all_warped3 = get_scrabble_board(all_frames3)

# # Show all frames for game 1
# for i in range(len(all_warped1)):
#     plt.imshow(all_warped1[i])
#     plt.axis("off")
#     plt.show()